In [55]:
import logging
import pandas as pd
import os
import numpy as np

In [56]:
from retrieve_post_by_userid import RetrievePostByUserId
t = RetrievePostByUserId(sessionid="")

In [57]:
def get_post_data(data,userid): 
    post_list = [] 
    if data is not None and data["data"] is not None and data["data"]["mediaData"] is not None and data["data"]["mediaData"]["edges"] is not None: 
        edges = data["data"]["mediaData"]["edges"] 
        if edges == []: 
            return [ 
                { 
                    "id": None, 
                    "post_pk": None, 
                    "caption_text": None, 
                    "like_count": None, 
                    "taken_at":None, 
                    "username": None, 
                    "user_pk": userid, 
                    "quote_count": None, 
                    "repost_count": None, 
                    "reshare_count": None, 
                } 
            ] 
        else: 
            try: 
                for edge in edges: 
                    thread_items = edge["node"]["thread_items"] 
                    for thread_item in thread_items: 
                        post = thread_item["post"] 
                        if post["caption"] is None or post["user"] is None or post["text_post_app_info"] is None: 
                            current_post = { 
                            "id": post.get("id",""), 
                            "post_pk": post.get("pk",""), 
                            "caption_text": None if post["caption"] is None else post.get("caption",{}).get("text",""), 
                            "like_count": post.get("like_count",""), 
                            "taken_at": post.get("taken_at",""), 
                            "username": None if post["user"] is None else post.get("user",{}).get("username",""), 
                            "user_pk": post.get("user",{}).get("pk",""), 
                            "quote_count": None if post["text_post_app_info"] is None else post["text_post_app_info"].get("quote_count",0), 
                            "repost_count": None if post["text_post_app_info"] is None else post["text_post_app_info"].get("repost_count",0), 
                            "reshare_count": None if post["text_post_app_info"] is None else post["text_post_app_info"].get("reshare_count",0), 
                        } 
                        else: 
                            current_post = { 
                                "id": post.get("id",""), 
                                "post_pk": post.get("pk",""), 
                                "caption_text": post.get("caption",{}).get("text",""), 
                                "like_count": post.get("like_count",""), 
                                "taken_at": post.get("taken_at",""), 
                                "username": post.get("user",{}).get("username",""), 
                                "user_pk": post.get("user",{}).get("pk",""), 
                                "quote_count": post["text_post_app_info"].get("quote_count",0), 
                                "repost_count": post["text_post_app_info"].get("repost_count",0), 
                                "reshare_count": post["text_post_app_info"].get("reshare_count",0), 
                            } 
                        post_list.append(current_post) 
                return post_list 
            except Exception as e: 
                raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}") 
    if data["errors"]: 
        for error in data["errors"]: 
            if error.get("message") == "execution error" and error.get("description") == "cannot see user": 
                return [ 
                        { 
                            "id": None, 
                            "post_pk": None, 
                            "caption_text": None, 
                            "like_count": None, 
                            "taken_at":None, 
                            "username": None, 
                            "user_pk": userid, 
                            "quote_count": None, 
                            "repost_count": None, 
                            "reshare_count": None, 
                        } 
                    ] 
    return []

def get_all_post(userid: str): 
    results = [] 
    try: 
        response = t.retrieve_thread_by_userid(userid) 
        d_out = get_post_data(response,userid) 
        results.extend(d_out) 
    except Exception as e: 
        print(response) 
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}") 
    return results

In [48]:
# df = pd.read_csv("data_chat_gpt.csv")
# df['user_pk'] = df['user_pk'].astype(str)
# unique_user_pk = df['user_pk'].unique()
# len(unique_user_pk)

In [58]:
df2 = pd.read_csv("followers_ChatGPT_3.csv")
df2['follower_pk'] = df2['follower_pk'].astype(str)
unique_user = df2['follower_pk'].unique()
len(unique_user)

4055

In [50]:
# unique_user = np.concatenate([unique_user_pk, unique_user_pk2])
# len(unique_user)

In [59]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "post_data_chatgpt3.csv"
processed_users = load_processed_users(csv_file)

In [60]:
len(processed_users)

1373

In [61]:
unique_user = unique_user[unique_user != "59075281376"]
unique_user = unique_user[unique_user != "1459172468"]
unique_user = unique_user[unique_user != "1631992202"]



In [62]:
import pandas as pd
i = len(processed_users)
for user in unique_user:
     if user not in processed_users:
        # print(user)
        i+=1
        posts = get_all_post(user)
        final_df = pd.DataFrame(posts)
        save_to_csv(final_df, csv_file)
        print(f"Processati {i} utenti su {len(unique_user)}")

print("Processo completato!")

Processati 1374 utenti su 4052
Processati 1375 utenti su 4052
Processati 1376 utenti su 4052
Processati 1377 utenti su 4052
Processati 1378 utenti su 4052
Processati 1379 utenti su 4052
Processati 1380 utenti su 4052
Processati 1381 utenti su 4052
Processati 1382 utenti su 4052
Processati 1383 utenti su 4052
Processati 1384 utenti su 4052
Processati 1385 utenti su 4052
Processati 1386 utenti su 4052
Processati 1387 utenti su 4052
Processati 1388 utenti su 4052
Processati 1389 utenti su 4052
Processati 1390 utenti su 4052
Processati 1391 utenti su 4052
Processati 1392 utenti su 4052
Processati 1393 utenti su 4052
Processati 1394 utenti su 4052
Processati 1395 utenti su 4052
Processati 1396 utenti su 4052
Processati 1397 utenti su 4052
Processati 1398 utenti su 4052
Processati 1399 utenti su 4052
Processati 1400 utenti su 4052
Processati 1401 utenti su 4052
Processati 1402 utenti su 4052
Processati 1403 utenti su 4052
Processati 1404 utenti su 4052
Processati 1405 utenti su 4052
Processa

RuntimeError: Errore generale nel recupero dei follower per l'utente 1499651689: 'data'

In [20]:
df1 = pd.read_csv(csv_file).drop_duplicates()
print(df1.shape)
df1

(49328, 10)


,id,post_pk,caption_text,like_count,taken_at,username,user_pk,quote_count,repost_count,reshare_count
0,3556503364705235589_57666094056,3.556503e+18,These cars just got roasted by ChatGPT! 🏎️😂,158.0,1.738188e+09,chatgptricks,57666094056,0.0,11.0,7.0
1,3555730866451646099_57666094056,3.555731e+18,Sam Altman’s first reaction to DeepSeek’s open...,71.0,1.738096e+09,chatgptricks,57666094056,0.0,2.0,10.0
2,3555632212571112619_57666094056,3.555632e+18,Meta AI’s latest upgrade allows it to remember...,8.0,1.738084e+09,chatgptricks,57666094056,0.0,0.0,NaN
3,3554870787011765397_57666094056,3.554871e+18,👉 Comment “Editor” and we’ll send you the link...,4.0,1.737994e+09,chatgptricks,57666094056,0.0,0.0,2.0
4,3554155796214168463_57666094056,3.554156e+18,From static to cinematic in one click 🎥\n\nRun...,73.0,1.737908e+09,chatgptricks,57666094056,0.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...
49323,3449431740732330571_651952269,3.449432e+18,NaN,5.0,1.725424e+09,hidaisyhi,651952269,0.0,0.0,NaN
49324,3443045977006356854_651952269,3.443046e+18,스친들아\n\n지난번 미국상사의 아들이 한국에 급히온다고\n원룸문의를 올린바있어\n...,5.0,1.724663e+09,hidaisyhi,651952269,0.0,0.0,NaN
49325,3443047645903199298_651952269,3.443048e+18,이어서\n새로운가족을 만났지만 너무나 위축되었는데\n인생의 첫도전을 한국에와서 시...,6.0,1.724663e+09,hidaisyhi,651952269,0.0,0.0,NaN
49326,3430607806956154513_651952269,3.430608e+18,스친들아\n\n급히 물어볼것이있어 지나치지말고\n의견구한다\n\n미국상사의 아들이 ...,12.0,1.723180e+09,hidaisyhi,651952269,0.0,1.0,NaN
